## Importing libraries to webscrape

In [1]:
from bs4 import BeautifulSoup
import requests
from time import sleep
import pickle

## Locating links to specific scripts in the html

In [3]:
url = 'https://www.imsdb.com/genre/Fantasy'

In [4]:
soup = BeautifulSoup(requests.get(url).content,'html.parser')
soup.select('a')[64:-7]

[<a href="/Movie Scripts/9 Script.html" title="9 Script">9</a>,
 <a href="/Movie Scripts/Addams Family, The Script.html" title="Addams Family, The Script">Addams Family, The</a>,
 <a href="/Movie Scripts/Aladdin Script.html" title="Aladdin Script">Aladdin</a>,
 <a href="/Movie Scripts/Antz Script.html" title="Antz Script">Antz</a>,
 <a href="/Movie Scripts/Army of Darkness Script.html" title="Army of Darkness Script">Army of Darkness</a>,
 <a href="/Movie Scripts/Batman Script.html" title="Batman Script">Batman</a>,
 <a href="/Movie Scripts/Batman 2 Script.html" title="Batman 2 Script">Batman 2</a>,
 <a href="/Movie Scripts/Batman and Robin Script.html" title="Batman and Robin Script">Batman and Robin</a>,
 <a href="/Movie Scripts/Batman Begins Script.html" title="Batman Begins Script">Batman Begins</a>,
 <a href="/Movie Scripts/Batman Forever Script.html" title="Batman Forever Script">Batman Forever</a>,
 <a href="/Movie Scripts/Beasts of the Southern Wild Script.html" title="Beasts o

## Initializing a pickle to store our scraped scripts

In [5]:
pickle.dump({},open('fan_scripts.p','wb'))

## Function to scrape scripts

In [6]:
def scrape_scripts(genre_url, counter_init=0, list_st=64,list_end=-7,pckl_file):
    """
    PARAMETERS
    ------------------
    genre_url (string):
        URL of the genre page with links to movies.
        Should have the form - "https://www.imsdb.com/genre/*Genre Name*"
    counter_init (integer):
        Counter to keep track of which movie you are up to. For display
        purposes only, does not impact the function of the code
    list_st (integer):
        Index to start at in the list of 'a' tags in the html.
    list_end (integer):
        Index to end at in the list of 'a' tags in the html.
    pckl_file (string):
        Where to pickle the scraped data to.
    """
    scripts = []
    # Get the list of movie script links from the main genre page.
    soup = BeautifulSoup(requests.get(genre_url).content,'html.parser')
    movie_links = [a['href'] for a in soup.select('a')[list_st:list_end]]
    counter = counter_init
    # For each movie in the list of links,
    #    -Navigate to the page with the script 
    #    -Gather the text of the script from the page
    #        -Remove blank lines and white spaces then join into a single string
    #    -Add this new script into the pickle.
    for movie_link in movie_links:
        sleep(0.5)
        movie = requests.get('https://www.imsdb.com'+movie_link)
        soup2 = BeautifulSoup(movie.content,'html.parser')
        script_url = soup2.select('table.script-details a')[-1]['href']
        # If the link isn't a script, skip to the next movie.
        if not script_url.startswith('/scripts/'):
            continue
        sleep(0.5)
        script = requests.get('https://www.imsdb.com'+script_url)
        soup3 = BeautifulSoup(script.content,'html.parser')
        # Different scripts have different tag formats. Try the most common ones.
        try:
            lines = [line.strip() for line in soup3.select('pre pre')[0].text.split('\n')]
            print(counter,'WORKED:',script_url)
        except IndexError:
            try:
                lines = [line.strip() for line in soup3.select('pre')[0].text.split('\n')]
                print(counter,'WORKED:',script_url)
            except IndexError:
                print(counter,'FAILED:',script_url)
                continue
        # Remove empty lines
        lines = [line for line in lines if line!='']
        # Load the existing pickle, add the new script with the title as the dictionary key,
        # and then pickle the new updated dictionary.
        temp = pickle.load(open(pckl_file,'rb'))
        try:
            temp[lines[0]] = ' '.join(lines)
        except IndexError:
            print('Scraping ended up returning nothing.')
            continue
        pickle.dump(temp,open(pckl_file,'wb'))
        
        sleep(0.5)
        counter+=1
    return scripts

In [7]:
scripts = scrape_scripts('https://www.imsdb.com/genre/Fantasy',0,64,-7,'fan_scripts.p')

0 WORKED: /scripts/9.html
1 WORKED: /scripts/Addams-Family,-The.html
2 WORKED: /scripts/Aladdin.html
3 WORKED: /scripts/Antz.html
4 WORKED: /scripts/Army-of-Darkness.html
5 WORKED: /scripts/Batman.html
6 WORKED: /scripts/Batman-2.html
7 WORKED: /scripts/Beasts-of-the-Southern-Wild.html
8 WORKED: /scripts/Beauty-and-the-Beast.html
9 WORKED: /scripts/Being-Human.html
10 WORKED: /scripts/Being-John-Malkovich.html
11 WORKED: /scripts/Big.html
12 WORKED: /scripts/Big-Fish.html
13 WORKED: /scripts/Blade-Trinity.html
14 WORKED: /scripts/Brazil.html
15 WORKED: /scripts/Bruce-Almighty.html
16 WORKED: /scripts/Butterfly-Effect,-The.html
17 WORKED: /scripts/Christ-Complex.html
18 WORKED: /scripts/Chronicles-of-Narnia-The-Lion,-the-Witch-and-the-Wardrobe.html
19 WORKED: /scripts/Cirque-du-Freak-The-Vampire's-Assistant.html
20 WORKED: /scripts/Clash-of-the-Titans.html
21 WORKED: /scripts/Conan-the-Barbarian.html
22 WORKED: /scripts/Constantine.html
23 WORKED: /scripts/Coraline.html
24 WORKED: /scri

In [8]:
len(scripts)

0

In [9]:
test = pickle.load(open('fan_scripts.p','rb'))

In [11]:
len([title.strip('"') for title in test.keys()])

116

In [12]:
len([title.strip('"') for title in test.keys()])

116